# BalticMaterials GmbH
## AI-Camera Luxonis OAK-1 MAX access
**Author:** Sven Nivera  
**Date:** 10.08.2023  
**Version:** 0.1.0  
**Description:** This notebook contains an example on how to access the OAK-1 MAX camera via USB using opencv and depthai.

In [1]:
import cv2
import depthai as dai
import numpy as np

In [2]:
pipeline = dai.Pipeline()

In [3]:
cam = pipeline.create(dai.node.ColorCamera) # add cam
cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_5312X6000) # 32MP
# cam.setNumFramesPool(2,2,1,1,1) usefull for minimizing RAM usage

In [4]:
xout_rgb = pipeline.create(dai.node.XLinkOut)
xout_rgb.setStreamName("rgb")
cam.preview.link(xout_rgb.input)

In [5]:
def frameNorm(frame, bbox):
    normVals = np.full(len(bbox), frame.shape[0])
    normVals[::2] = frame.shape[1]
    return (np.clip(np.array(bbox), 0, 1) * normVals).astype(int)

Activation of logging is only usefull in a .py-excecution. There will be no output in a notebook.

In [ ]:
with dai.Device(pipeline) as device: # initialize device via USB3
    device.setLogLevel(dai.LogLevel.INFO)
    device.setLogOutputLevel(dai.LogLevel.INFO)
    q_rgb = device.getOutputQueue("rgb") # frame queue for host side filling up with frames from the cam
    frame = None
    while True:
        in_rgb = q_rgb.tryGet() # Consuming latest (or None) frame from queue
        if in_rgb is not None:
            frame = in_rgb.getCvFrame() # Receive frame 
            if frame is not None:
                cv2.imshow("preview", frame)
                if cv2.waitKey(1) == ord('q'):
                    break

: 